In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('houseprices_modified.csv')
df

,LotFrontageSF,LotAreaSF,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMaterial,...,GrLivArea,GarageType,GarageYrBlt,GarageCars,GarageSF,GarageQual,WoodDeckSF,OpenPorchSF,MiscFeature,SalePrice
0,65.0,8450,Single Family Detached,2 Story,7,5,2003,2003,Gable,Standard Composite Shingle,...,1710,Attached,2003,2,548,3,0,61,None,208500
1,80.0,9600,Single Family Detached,1 Story,6,8,1976,1976,Gable,Standard Composite Shingle,...,1262,Attached,1976,2,460,3,298,0,None,181500
2,68.0,11250,Single Family Detached,2 Story,7,5,2001,2002,Gable,Standard Composite Shingle,...,1786,Attached,2001,2,608,3,0,42,None,223500
3,60.0,9550,Single Family Detached,2 Story,7,5,1915,1970,Gable,Standard Composite Shingle,...,1717,Detached,1998,3,642,3,0,35,None,140000
4,84.0,14260,Single Family Detached,2 Story,8,5,2000,2000,Gable,Standard Composite Shingle,...,2198,Attached,2000,3,836,3,192,84,None,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1451,62.0,7917,Single Family Detached,2 Story,6,5,1999,2000,Gable,Standard Composite Shingle,...,1647,Attached,1999,2,460,3,0,40,None,175000
1452,85.0,13175,Single Family Detached,1 Story,6,6,1978,1988,Gable,Standard Composite Shingle,...,2073,Attached,1978,2,500,3,349,0,None,210000
1453,66.0,9042,Single Family Detached,2 Story,7,9,1941,2006,Gable,Standard Composite Shingle,...,2340,Attached,1941,1,252,3,0,60,Shed,266500
1454,68.0,9717,Single Family Detached,1 Story,5,6,1950,1996,Hip,Standard Composite Shingle,...,1078,Attached,1950,1,240,3,366,0,None,142125


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1456 entries, 0 to 1455
Data columns (total 32 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   LotFrontageSF  1197 non-null   float64
 1   LotAreaSF      1456 non-null   int64  
 2   BldgType       1456 non-null   object 
 3   HouseStyle     1456 non-null   object 
 4   OverallQual    1456 non-null   int64  
 5   OverallCond    1456 non-null   int64  
 6   YearBuilt      1456 non-null   int64  
 7   YearRemodAdd   1456 non-null   int64  
 8   RoofStyle      1456 non-null   object 
 9   RoofMaterial   1456 non-null   object 
 10  ExterQual      1456 non-null   int64  
 11  ExterCond      1456 non-null   int64  
 12  Foundation     1456 non-null   object 
 13  BsmtQual       1456 non-null   int64  
 14  BsmtCond       1456 non-null   int64  
 15  BsmtFinSF      1456 non-null   int64  
 16  TotalBsmtSF    1456 non-null   int64  
 17  Heating        1456 non-null   object 
 18  HeatingQ

In [4]:
df.describe()

,LotFrontageSF,LotAreaSF,OverallQual,OverallCond,YearBuilt,YearRemodAdd,ExterQual,ExterCond,BsmtQual,BsmtCond,...,FirstFlrSF,SecondFlrSF,GrLivArea,GarageYrBlt,GarageCars,GarageSF,GarageQual,WoodDeckSF,OpenPorchSF,SalePrice
count,1197.000000,1456.000000,1456.000000,1456.000000,1456.00000,1456.000000,1456.000000,1456.000000,1456.00000,1456.000000,...,1456.000000,1456.000000,1456.000000,1456.000000,1456.000000,1456.000000,1456.000000,1456.000000,1456.000000,1456.000000
mean,69.693400,10448.784341,6.088599,5.576236,1971.18544,1984.819368,3.392170,3.083791,3.48489,2.934753,...,1157.108516,343.532967,1506.502060,1976.439560,1.764423,471.568681,2.809753,93.833791,46.221154,180151.233516
std,23.054474,9860.763449,1.369669,1.113966,30.20159,20.652143,0.570206,0.351509,0.87409,0.552906,...,369.307331,431.528915,496.815378,26.308749,0.746215,211.986421,0.723822,125.192349,65.352424,76696.592530
min,21.000000,1300.000000,1.000000,1.000000,1872.00000,1950.000000,2.000000,1.000000,0.00000,0.000000,...,334.000000,0.000000,334.000000,1872.000000,0.000000,0.000000,0.000000,0.000000,0.000000,34900.000000
25%,59.000000,7538.750000,5.000000,5.000000,1954.00000,1966.750000,3.000000,3.000000,3.00000,3.000000,...,882.000000,0.000000,1128.000000,1959.000000,1.000000,329.500000,3.000000,0.000000,0.000000,129900.000000
50%,69.000000,9468.500000,6.000000,5.000000,1972.00000,1993.500000,3.000000,3.000000,4.00000,3.000000,...,1086.000000,0.000000,1458.500000,1978.000000,2.000000,478.500000,3.000000,0.000000,24.000000,163000.000000
75%,80.000000,11588.000000,7.000000,6.000000,2000.00000,2004.000000,4.000000,3.000000,4.00000,3.000000,...,1389.250000,728.000000,1775.250000,2001.000000,2.000000,576.000000,3.000000,168.000000,68.000000,214000.000000
max,313.000000,215245.000000,10.000000,9.000000,2010.00000,2010.000000,5.000000,5.000000,5.00000,4.000000,...,3228.000000,1818.000000,3627.000000,2010.000000,4.000000,1390.000000,5.000000,857.000000,547.000000,625000.000000


In [5]:
df['BldgType'].value_counts()

Single Family Detached    1216
Townhouse End Unit         114
Duplex                      52
Townhouse                   43
Two Family Conversion       31
Name: BldgType, dtype: int64

In [6]:
df['HouseStyle'].value_counts()

1 Story        726
2 Story        441
1.5 Story      168
Split Level     65
Split Foyer     37
2.5 Story       19
Name: HouseStyle, dtype: int64

In [7]:
df['RoofStyle'].value_counts()

Gable      1140
Hip         283
Flat         13
Gambrel      11
Mansard       7
Shed          2
Name: RoofStyle, dtype: int64

In [8]:
df['RoofMaterial'].value_counts()

Standard Composite Shingle    1432
Tar & Gravel                    11
Wood Shingles                    5
Wood Shakes                      5
Metal                            1
Membrane                         1
Roll                             1
Name: RoofMaterial, dtype: int64

In [10]:
df['Foundation'].value_counts()

Poured Concrete    643
Cinder Block       634
Brick & Tile       146
Slab                24
Stone                6
Wood                 3
Name: Foundation, dtype: int64

In [11]:
df['Heating'].value_counts()

GasA     1424
GasW       18
Grav        7
Wall        4
OthW        2
Floor       1
Name: Heating, dtype: int64

In [12]:
df.columns

Index(['LotFrontageSF', 'LotAreaSF', 'BldgType', 'HouseStyle', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMaterial',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtFinSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir',
       'FirstFlrSF', 'SecondFlrSF', 'GrLivArea', 'GarageType', 'GarageYrBlt',
       'GarageCars', 'GarageSF', 'GarageQual', 'WoodDeckSF', 'OpenPorchSF',
       'MiscFeature', 'SalePrice'],
      dtype='object')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1456 entries, 0 to 1455
Data columns (total 32 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   LotFrontageSF  1197 non-null   float64
 1   LotAreaSF      1456 non-null   int64  
 2   BldgType       1456 non-null   object 
 3   HouseStyle     1456 non-null   object 
 4   OverallQual    1456 non-null   int64  
 5   OverallCond    1456 non-null   int64  
 6   YearBuilt      1456 non-null   int64  
 7   YearRemodAdd   1456 non-null   int64  
 8   RoofStyle      1456 non-null   object 
 9   RoofMaterial   1456 non-null   object 
 10  ExterQual      1456 non-null   int64  
 11  ExterCond      1456 non-null   int64  
 12  Foundation     1456 non-null   object 
 13  BsmtQual       1456 non-null   int64  
 14  BsmtCond       1456 non-null   int64  
 15  BsmtFinSF      1456 non-null   int64  
 16  TotalBsmtSF    1456 non-null   int64  
 17  Heating        1456 non-null   object 
 18  HeatingQ

In [14]:
BldgType_le = LabelEncoder()
df['BldgType_Encoded'] = BldgType_le.fit_transfsorm(df['BldgType'])

In [15]:
HouseStyle_le = LabelEncoder()
df['HouseStyle_Encoded'] = HouseStyle_le.fit_transform(df['HouseStyle'])

In [16]:
RoofStyle_le = LabelEncoder()
df['RoofStyle_Encoded'] = RoofStyle_le.fit_transform(df['RoofStyle'])

In [17]:
RoofMaterial_le = LabelEncoder()
df['RoofMaterial_Encoded'] = RoofMaterial_le.fit_transform(df['RoofMaterial'])

In [18]:
Foundation_le = LabelEncoder()
df['Foundation_Encoded'] = Foundation_le.fit_transform(df['Foundation'])

In [19]:
Heating_le = LabelEncoder()
df['Heating_Encoded'] = Heating_le.fit_transform(df['Heating'])